In [ ]:
from scapy.all import sniff, conf
import csv
from datetime import datetime

'''
conda activate netsec_env
spyder
'''

# scapy layer-3 socket config for sniffing
conf.l3socket = conf.L3socket

# csv generation
CSV_FILE = 'netlog.csv'

# generate header
try:
    with open(CSV_FILE, mode='x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            "timestamp", "src_ip", "dst_ip", "protocol", "length",
            "src_port", "dst_port", "flags", "ttl", "payload_size", "protocol_name"
        ])
except FileExistsError:
    pass

def packet_callback(packet):
    try:
        # fetch timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if packet.haslayer('IP'):
            src_ip = packet['IP'].src
            dst_ip = packet['IP'].dst
            proto = packet.proto
            length = len(packet)
            ttl = packet['IP'].ttl  # Time to Live

            # assign protocol name based on packet protocol
            if proto == 1:
                protocol_name = 'ICMP'
            elif proto == 6:
                protocol_name = 'TCP'
            elif proto == 17:
                protocol_name = 'UDP'
            else:
                protocol_name = 'Other'

            # init for default TCP/UDP data
            src_port = dst_port = flags = payload_size = None

            # if TCP packet, extract port numbers and flags
            if packet.haslayer('TCP'):
                src_port = packet['TCP'].sport
                dst_port = packet['TCP'].dport
                flags = packet['TCP'].flags
                payload_size = len(packet['TCP'].payload)

            # if UDP packet, extract port numbers (no flags)
            elif packet.haslayer('UDP'):
                src_port = packet['UDP'].sport
                dst_port = packet['UDP'].dport
                flags = None
                payload_size = len(packet['UDP'].payload)

            # write to log
            with open(CSV_FILE, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    timestamp, src_ip, dst_ip, proto, length,
                    src_port, dst_port, flags, ttl, payload_size, protocol_name
                ])
    except Exception as e:
        print(f"Error processing packet: {e}")

print("Sniffing started... Press Ctrl+C to stop.")
try:
    sniff(prn=packet_callback, store=False)  # sniffing packets
except KeyboardInterrupt:
    print("\nSniffing stopped.")
